In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.3

In [2]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"thnhnguyenphuc","key":"c4567d6a802976fa686fb1f293c5ce5f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi

# https://technowhisp.com/kaggle-api-python-documentation/
# Authenticate to Kaggle
api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('duyminhnguyentran/csc15105', unzip=True)

In [4]:
file_path_train = "Project2_Data/train.json"
file_path_test = "Project2_Data/test.json"

In [5]:
import json

with open(file_path_train, "r") as read_file:
    train_data = json.load(read_file)

with open(file_path_test, "r") as read_file:
    test_data = json.load(read_file)

In [6]:
for item in train_data:
    for qas in item['qas']:
        if qas['answers'][0]['answer_start'] == -1:
            qas['answers'][0]['answer_start'] = item['context'].find(qas['answers'][0]['text'])
    item['qas'] = [qas for qas in item['qas'] if len(qas['answers'][0]['text']) >= 1 or qas['answers'][0]['text'].isdigit()]
    item['qas'] = [qas for qas in item['qas'] if len(qas['question']) >= 1]
with open('modified_train_data.json', 'w') as file:
    json.dump(train_data, file, indent=4)

for item in test_data:
    for qas in item['qas']:
        if qas['answers'][0]['answer_start'] == -1:
            qas['answers'][0]['answer_start'] = item['context'].find(qas['answers'][0]['text'])
    item['qas'] = [qas for qas in item['qas'] if len(qas['answers'][0]['text']) >= 1 or qas['answers'][0]['text'].isdigit()]
    item['qas'] = [qas for qas in item['qas'] if len(qas['question']) >= 1]
with open('modified_test_data.json', 'w') as file:
    json.dump(test_data, file, indent=4)

In [7]:
file_path_train = "modified_train_data.json"
file_path_test = "modified_test_data.json"

In [8]:
import json

with open(file_path_train, "r") as read_file:
    train = json.load(read_file)

with open(file_path_test, "r") as read_file:
    test = json.load(read_file)

In [9]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [10]:
model_type="electra"
model_name= "google/electra-base-discriminator"


In [11]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5

In [12]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "train_batch_size": 64,
    "eval_batch_size": 32,
}


In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 49322/49322 [00:00<00:00, 792443.97it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 1 of 5:   0%|          | 0/808 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/808 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

(4040,
 {'global_step': [808, 1000, 1616, 2000, 2424, 3000, 3232, 4000, 4040],
  'correct': [1014, 1202, 1578, 1619, 1790, 1883, 1980, 2072, 2069],
  'similar': [10576, 9558, 7831, 9097, 7930, 8861, 8014, 8548, 8559],
  'incorrect': [4180, 5010, 6361, 5054, 6050, 5026, 5776, 5150, 5142],
  'train_loss': [3.043097972869873,
   3.2189865112304688,
   2.9277701377868652,
   2.8985595703125,
   2.7689712047576904,
   2.8763504028320312,
   2.2886838912963867,
   2.9617204666137695,
   2.319939136505127],
  'eval_loss': [-1.8692180699951644,
   -1.7664542281189555,
   -1.5736020233921664,
   -2.006339156189555,
   -1.792494635517408,
   -1.9021841377538684,
   -2.0751178675048356,
   -2.2128547358558994,
   -2.2000234223887816]})

In [15]:
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/517 [00:00<?, ?it/s]

In [16]:
result

{'correct': 2069,
 'similar': 8559,
 'incorrect': 5142,
 'eval_loss': -2.2000234223887816}

In [17]:
texts

{'correct_text': {'5733bf84d058e614000b61bd': 'ba',
  '5733bf84d058e614000b61be': 'Tháng Chín năm 1876,',
  '5733a6424776f41900660f4e': 'năm 1920,',
  '5733a7bd4776f41900660f6b': '1924',
  '5733ac31d058e614000b5ff4': '1986',
  '5733ad384776f41900660fee': 'năm 1963,',
  '5733ae924776f41900661014': '3577',
  '5733afd3d058e614000b6045': '18 tổng',
  '5733afd3d058e614000b6046': 'USA Today $ $$ xếp Notre Dame 10 tổng thể cho các trường đại học Mỹ dựa trên dữ liệu từ Cao đẳng Thực tế. Forbes. Best Colleges của com của Mỹ đứng thứ Notre Dame thứ 13 trong số các trường đại học tại Hoa Kỳ vào năm 2015,',
  '5733b2fe4776f41900661090': '1928',
  '5733b3d64776f419006610a5': '44',
  '5733b5344776f419006610dd': 'John Jenkins,',
  '5733b5df4776f41900661109': '12.179',
  '5733b5df4776f41900661105': '8448',
  '5733b5df4776f41900661106': '21-24%',
  '5733b699d058e614000b611a': '15',
  '5733b699d058e614000b611c': '14',
  '57338a51d058e614000b5cf3': '1883',
  '573394c84776f41900660ddf': '1240',
  '5733974

In [18]:
# Make predictions with the model
to_predict = [
    {
        "context": "Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất. Kể từ thời điểm chuyển giao thế kỷ 21, đã có một xu hướng phát triển của một ngành công nghiệp tài nguyên tái tạo và giảm cả tiêu thụ và sử dụng tài nguyên nhiên liệu hóa thạch. Năm 2006, nhà máy điện năng lượng mặt trời lớn nhất thế giới tại thời điểm đó, nhà máy điện Moura quang điện, bắt đầu hoạt động gần Moura, ở phía nam, trong khi trang trại năng lượng sóng thương mại đầu tiên trên thế giới, Aguçadoura sóng trại , mở trong khu vực Norte  (2008). Đến cuối năm 2006, 66% sản lượng điện của nước này là từ nhà máy than và điện nhiên liệu, trong khi 29% được lấy từ đập thủy điện, và 6% bằng năng lượng gió. .",
        "qas": [
            {
                "question": "hai nguồn đáng kể nhất của năng lượng tái tạo ở Bồ Đào Nha là gì?",
                "id": "0",
            }
        ],
    }
]

In [19]:
answers, probabilities = model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1271.00it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['Bồ Đào Nha có nguồn tài nguyên đáng kể gió và điện sông, hai nguồn năng lượng tái tạo hiệu quả nhất.', 'nhất.', 'quả nhất.']}]
